# SubPTJ crawling

## Crawling

In [1]:
import time
import json
import urllib
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

### HRD-net URL
---
한 페이지에서 원하는 데이터 뽑아서 모든 페이지 적용할 예정

In [2]:
url = 'https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000286660&tracseTme=1&crseTracseSe=C0061&trainstCstmrId=500020018488#undefined'

resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')

> 기관명

In [3]:
# 기관명
for organ in soup.select('p.add'):
    print(organ.text.strip())
#     data.append(organ.text.strip())

스마트인재개발원


> 과정명

In [4]:
# 과정명
for course in soup.select('h4.tit'):
    print(course.text.replace('\n','').replace('\t',''))
#     data.append(course.text.replace('\n','').replace('\t',''))

빅데이터 분석서비스 개발자과정(NCS)모집마감


> 훈련비

In [5]:
# 훈련비
for price in soup.select('p.price'):
    print(price.text.strip().replace('\n','')[:price.text.find('원')].strip().replace('\t',''))
#     data.append(price.text.strip().replace('\n','')[:price.text.find('원')].strip().replace('\t',''))

13,762,240 원



In [6]:

for i in soup.select('span.con'):
    if i.text.find('일') != -1:
        date = i.text.replace('\n,','').replace('\t','').strip().replace('\n','')
        print(f'date : {date}')
    elif i.text.find('세') != -1:
        age = i.text.replace('\n,','').replace('\t','').strip()
        print('age : ' + age)
    elif i.text.find(')') != -1:
        if  i.text.find('~') == -1:
            NCS = i.text.replace('\n,','').replace('\t','').strip()
            print('NCS : ' + NCS[:NCS.find('(')])
        else:
            print('datetime : ' + i.text.replace('\n,','').replace('\t','').strip())
    
#         data.append(i.text.replace('\n,','').replace('\t','').strip())
#     print(i.text.replace('\n', '').replace('\t','').strip())

NCS : 응용SW엔지니어링
datetime : 2020-11-02
~2021-04-12
(1회차)
date : 116일,총928시간
age : 27 세


In [7]:
# NCS 자격분류
for NCS in soup.select('span.con'):
    if re.findall('\d{6,}',NCS.text) != []:
        print(re.findall('.+\d{6,}.',NCS.text).pop().replace('\t',''))

응용SW엔지니어링(20010202)


> span 태그가 적절하지 않음 -> 쓸데없는게 너무 뽑힘

> 근데 대안도 없다 이걸로 가자

In [8]:
# 수강생 평균 만족도
soup.select('div.starRating')[0].get('aria-label')

'만족도 100점 기준86.88점'

In [9]:
# 훈련시간
for i in soup.select('span.con'):
    if i.text.find('일') != -1:
        print(i.text.replace('\n,','').replace('\t','').strip())
#         data.append(i.text.replace('\n,','').replace('\t','').strip())

116일,
총928시간


> table.view 태그로 회차별 정보 한 번에 뽑기

In [10]:
# 전체 회차
len(soup.select('a.tool_tip_btn'))

14

In [11]:
# 회차별 만족도 
count = len(soup.select('a.tool_tip_btn')) + 1
for i in soup.select('table.view')[2:]:
    if i.text.find('훈련진행중') != -1:
        count -= 1
        print(f'{count}회차 : 훈련진행중')
    else:
        for j in i.select('td'):
            tableview = j.text.replace('\n','').replace('\t','').strip()
            if re.findall('[.\d]+\s점',tableview) != []:
                count -= 1
                print(f"{count}회차 : " + re.findall('[.\d]+\s점',tableview).pop().replace(' 점',''))

14회차 : 훈련진행중
13회차 : 훈련진행중
12회차 : 훈련진행중
11회차 : 훈련진행중
10회차 : 75.2
9회차 : 82.4
8회차 : 90.84
7회차 : 90
6회차 : 91.58
5회차 : 84.54
4회차 : 89.6
3회차 : 89.56
2회차 : 80.74
1회차 : 95.56


In [12]:
# 회차별 모집인원
count = len(soup.select('a.tool_tip_btn')) + 1
for i in soup.select('table.view')[2:]:
    if i.text.find('훈련진행중') != -1:
        count -= 1
        print(f'{count}회차 : 훈련진행중')
    else:
        for j in i.select('td'):
            if re.findall('\d+명/',j.text) != []:
                count -= 1
                print(f"{count}회차 : " + re.findall('\d+명/',j.text).pop().replace('/',''))

14회차 : 훈련진행중
13회차 : 훈련진행중
12회차 : 훈련진행중
11회차 : 훈련진행중
10회차 : 25명
9회차 : 25명
8회차 : 25명
7회차 : 25명
6회차 : 24명
5회차 : 25명
4회차 : 25명
3회차 : 25명
2회차 : 28명
1회차 : 28명


> 망한 코드

In [13]:
# ?????????????????????
# [(\d]+/ : 취업인원
# /\d+ : 수료인원
count = len(soup.select('a.tool_tip_btn')) + 1
break_count = 2 
comple = ''
entire = ''
for i in soup.select('table.view'):
    if i.text.find('훈련진행중') != -1:
        count -= 1
        print(f'{count}회차 : 훈련진행중')
    else:
        for j in i.select('td'):
            break_count += 1
            if break_count % 3 == 0:
                if re.findall('\(/\)',j.text) != []:
                    count -= 1
                    print(f'{count}회차 : '+ re.findall('\(/\)',j.text).pop().strip('(').strip(')'))
#                     print('-')
#                     print('-')
                elif re.findall('[(\d]+/', j.text) != [] and re.findall('\d+\)', j.text) != []:
                    count -= 1
                    print(f'{count}회차 : '+ re.findall('[(\d]+/', j.text).pop().replace('(','') + re.findall('\d+\)', j.text).pop().replace(')',''))
#                     print(re.findall('[(\d]+/', j.text).pop().replace('(','')[:re.findall('[(\d]+/', j.text).pop().replace('(','').find('/')])
#                     print(re.findall('\d+\)', j.text).pop().replace(')','')[re.findall('\d+\)', j.text).pop().replace(')','').find('/')+1:])
#                     if re.findall('[(\d]+/', j.text) != []: # 훈련시작일 
#                         count -= 1
#                         comple = (re.findall('[(\d]+/', j.text).pop()) 
#                     if re.findall('\d+\)', j.text) != []:
#                         count -= 1
#                         entire = (re.findall('\d+\)', j.text).pop())
#                     print(f'{count}회차 : ',comple, entire)
            else:
                pass

14회차 : 훈련진행중
13회차 : 훈련진행중
12회차 : 훈련진행중
11회차 : 훈련진행중
10회차 : /
9회차 : /
8회차 : /23
7회차 : 20/23
6회차 : 1/19
5회차 : 15/18
4회차 : 14/25
3회차 : 2/23
2회차 : 16/21
1회차 : 21/26


> 다시 짠 코드

In [23]:
# 취업률 (고용보험가입 3개월) 취업인원/수료인원
empl_rate_3 = [] # 회차별 취업률(3개월)
empl_3 = [] # 회차별 취업인원 (3개월)
comp_3 = [] # 회차별 수료인원 (3개월)
series_count = len(soup.select('a.tool_tip_btn'))
# count = len(soup.select('a.tool_tip_btn')) + 1
cnt = 0
for i in soup.select('table.view'):
    if i.text.find('취업률 고용보험가입 3개월') != -1:
        for j in i.select('b'):
            cnt += 1
            if cnt % 3 == 1:
                rate_empl = j.text.replace('\n','').replace('\t','')
                empl_rate_3.append(rate_empl[:rate_empl.find("%")])
                empl_3.append(rate_empl[rate_empl.find("%")+2:rate_empl.find("/")])
                comp_3.append(rate_empl[rate_empl.find("/")+1:-1])
#                 print(f'취업률 : {rate_empl[:rate_empl.find("%")]}')
#                 print(f'취업인원 : {rate_empl[rate_empl.find("%")+2:rate_empl.find("/")]}')
#                 print(f'수료인원 : {rate_empl[rate_empl.find("/")+1:-1]}')

while len(empl_rate_3) != series_count:
    empl_rate_3.insert(0,'훈련진행중')
while len(empl_3) != series_count:
    empl_3.insert(0,'훈련진행중')
while len(comp_3) != series_count:
    comp_3.insert(0,'훈련진행중')
print(empl_rate_3)
print(empl_3)
print(comp_3)
len(empl_rate_3), len(empl_3), len(comp_3)

['훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '', '', '60.9', '87', '63.2', '0', '56', '73.9', '57.1', '80.8']
['훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '', '', '14', '20', '12', '0', '14', '17', '12', '21']
['훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '', '', '23', '23', '19', '18', '25', '23', '21', '26']


(14, 14, 14)

In [24]:
# 취업률 (고용보험가입 6개월) 취업인원/수료인원
empl_rate_6 = [] # 회차별 취업률(6개월)
empl_6 = [] # 회차별 취업인원 (6개월)
comp_6 = [] # 회차별 수료인원 (6개월)
series_count = len(soup.select('a.tool_tip_btn'))
# count = len(soup.select('a.tool_tip_btn')) + 1
cnt = 0
for i in soup.select('table.view'):
    if i.text.find('취업률 고용보험가입 6개월') != -1:
        for j in i.select('b'):
            cnt += 1
            if cnt % 3 == 2:
                rate_empl = j.text.replace('\n','').replace('\t','')
                empl_rate_6.append(rate_empl[:rate_empl.find("%")])
                empl_6.append(rate_empl[rate_empl.find("%")+2:rate_empl.find("/")])
                comp_6.append(rate_empl[rate_empl.find("/")+1:-1])
#                 print(f'취업률 : {rate_empl[:rate_empl.find("%")]}')
#                 print(f'취업인원 : {rate_empl[rate_empl.find("%")+2:rate_empl.find("/")]}')
#                 print(f'수료인원 : {rate_empl[rate_empl.find("/")+1:-1]}')

while len(empl_rate_6) != series_count:
    empl_rate_6.insert(0,'훈련진행중')
while len(empl_6) != series_count:
    empl_6.insert(0,'훈련진행중')
while len(comp_6) != series_count:
    comp_6.insert(0,'훈련진행중')
print(empl_rate_6)
print(empl_6)
print(comp_6)
len(empl_rate_6), len(empl_6), len(comp_6)

['훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '', '', '', '', '84.2', '83.3', '72', '82.6', '76.2', '92.3']
['훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '', '', '', '', '16', '15', '18', '19', '16', '24']
['훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '', '', '23', '23', '19', '18', '25', '23', '21', '26']


(14, 14, 14)

In [16]:
count = len(soup.select('a.tool_tip_btn')) + 1
for series in soup.select('span.con')[17:]:
    if series.text.find('~') != -1:
        count -= 1
        print(f'{count}회차 : ')
        print(series.text.replace('\n','').replace('\t','').strip()[:series.text.replace('\n','').replace('\t','').strip().find('~')])
        print(series.text.replace('\n','').replace('\t','').strip()[series.text.replace('\n','').replace('\t','').strip().find('~')+1:])

14회차 : 
2022-05-02 
2022-10-18
13회차 : 
2022-03-14 
2022-08-24
12회차 : 
2022-02-15 
2022-08-04
11회차 : 
2022-02-15 
2022-08-04
10회차 : 
2022-01-10 
2022-06-23
9회차 : 
2021-12-01 
2022-05-17
8회차 : 
2021-11-15 
2022-04-27
7회차 : 
2021-09-15 
2022-02-28
6회차 : 
2021-08-25 
2022-01-28
5회차 : 
2021-04-01 
2021-08-31
4회차 : 
2021-03-17 
2021-08-19
3회차 : 
2021-03-15 
2021-08-13
2회차 : 
2020-12-07 
2021-05-21
1회차 : 
2020-11-02 
2021-04-12


In [17]:
# 수강확정인원, 수강신청인원
for a in soup.select('table.view'):
    for b in a.select('td'):
        if b.text.find('수강확정인원') != -1:
            print(re.findall('\d+',b.text))

['25', '25']
['25', '25']
['25', '25']
['25', '25']
['25', '25']
['25', '25']
['25', '25']
['25', '25']
['24', '24']
['25', '25']
['25', '25']
['25', '25']
['28', '28']
['28', '28']


### 한 페이지에서 얻을 수 있는 모든 정보 얻음
---
- 회차별 만족도

- 회차별 모집인원

- 회차별 취업인원/수료인원

- 훈련시작일 ~훈련종료일

- 수강확정인원, 수강신청인원

### 페이지 내 링크 통해 찾을 수 있는 데이터
---
> N회차 보러가기 링크 들어가서 안에서 찾을 수 있는 것을 찾기
- 기관명
- 과정명
- 훈련비
- 훈련기간

1. N회차 링크 따기 ok

> column 확인용
1. 기관명 ok
2. 과정명 ok
3. 훈련비(원) ok
4. 훈련시간 ok
5. NCS 직무분류 ok
6. 수강생 평균 연령대(세) ok
7. 수강생 평균 만족도 ok
8. 회차 ok
9. 훈련기간 ok
10. 훈련시작일 ok
11. 훈련종료일 ok
12. 모집인원 ok
13. 수강확정인원 ok
14. 수강신청인원 ok
15. 회차별 만족도 ok
16. 취업률 고용보험가입 3개월 ok
17. 취업인원 ok
18. 수료인원 ok

## 1차 페이지 탐색 크롤링 시작

In [18]:
# 회차별 만족도
print('-'*15,'회차별 만족도','-'*15)
count = len(soup.select('a.tool_tip_btn')) + 1
for i in soup.select('table.view')[2:]:
    if i.text.find('훈련진행중') != -1:
        count -= 1
        print(f'{count}회차 : 훈련진행중')
    else:
        for j in i.select('td'):
            tableview = j.text.replace('\n','').replace('\t','').strip()
            if re.findall('[.\d]+\s점',tableview) != []:
                count -= 1
                print(f"{count}회차 : " + re.findall('[.\d]+\s점',tableview).pop().replace(' 점',''))

# 회차별 훈련기간
print('-'*15,'회차별 훈련기간','-'*15)
count = len(soup.select('a.tool_tip_btn')) + 1
for series in soup.select('span.con')[17:]:
    if series.text.find('~') != -1:
        count -= 1
        print(f'{count}회차 : ' + series.text.replace('\n','').replace('\t','').strip())
        
# 회차별 모집인원
print('-'*15,'회차별 모집인원','-'*15)
count = len(soup.select('a.tool_tip_btn')) + 1
for i in soup.select('table.view')[2:]:
    if i.text.find('훈련진행중') != -1:
        count -= 1
        print(f'{count}회차 : 훈련진행중')
    else:
        for j in i.select('td'):
            if re.findall('\d+명/',j.text) != []:
                count -= 1
                print(f"{count}회차 : " + re.findall('\d+명/',j.text).pop().replace('/',''))


# 회차별 수료, 취업 인원
print('-'*15,'회차별 [수료, 취업]인원','-'*15)
# [(\d]+/ : 취업인원
# /\d+ : 수료인원
count = len(soup.select('a.tool_tip_btn')) + 1
break_count = 2 
comple = ''
entire = ''
for i in soup.select('table.view'):
    if i.text.find('훈련진행중') != -1:
        count -= 1
        print(f'{count}회차 : 훈련진행중')
    else:
        for j in i.select('td'):
            break_count += 1
            if break_count % 3 == 0:
                if re.findall('\(/\)',j.text) != []:
                    count -= 1
                    print(f'{count}회차 : ' +re.findall('\(/\)',j.text).pop())
                elif re.findall('[(\d]+/', j.text) != [] and re.findall('\d+\)', j.text) != []:
                    count -= 1
                    print(f'{count}회차 : '+(re.findall('[(\d]+/', j.text).pop().replace('(','')) + re.findall('\d+\)', j.text).pop().replace(')',''))
            else:
                pass
            
# 회차별 수강확정인원, 수강신청인원
print('-'*15,'회차별 수강 [확정,신청] 인원','-'*15)
for a in soup.select('table.view'):
    for b in a.select('td'):
        if b.text.find('수강확정인원') != -1:
            print(re.findall('\d+',b.text))
for index, patterns in enumerate(soup.select('a.tool_tip_btn')):
    series_url = f'https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000286660&tracseTme={index+1}&crseTracseSe=C0061&trainstCstmrId=500020018488#undefined'
    resp_series = requests.get(series_url)
    soup_series = BeautifulSoup(resp_series.text, "html.parser")
    
    print(f'----------------{index+1} 회차 ---------------')
    
    # 차단 방지 시간 딜레이
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(5)
    time.sleep(a)
    
    # 기관명
    for organ in soup_series.select('p.add'):
        print(organ.text.strip())
#     data.append(organ.text.strip())
    
    # 과정명
    for course in soup_series.select('h4.tit'):
        print(course.text.replace('\n','').replace('\t',''))
#     data.append(course.text.replace('\n','').replace('\t',''))
    
    # 훈련비
    for price in soup_series.select('p.price'):
        print(price.text.strip().replace('\n','')[:price.text.find('원')].strip().replace('\t',''))
#     data.append(price.text.strip().replace('\n','')[:price.text.find('원')].strip().replace('\t',''))

    # 훈련시간
    for i in soup_series.select('span.con'):
        if i.text.find('일') != -1:
            print(i.text.replace('\n,','').replace('\t','').strip())
#             data.append(i.text.replace('\n,','').replace('\t','').strip())
    # NCS 자격분류
    for NCS in soup_series.select('span.con'):
        if re.findall('\d{6,}',NCS.text) != []:
            print(re.findall('.+\d{6,}.',NCS.text).pop().replace('\t',''))


--------------- 회차별 만족도 ---------------
14회차 : 훈련진행중
13회차 : 훈련진행중
12회차 : 훈련진행중
11회차 : 훈련진행중
10회차 : 75.2
9회차 : 82.4
8회차 : 90.84
7회차 : 90
6회차 : 91.58
5회차 : 84.54
4회차 : 89.6
3회차 : 89.56
2회차 : 80.74
1회차 : 95.56
--------------- 회차별 훈련기간 ---------------
14회차 : 2022-05-02 ~2022-10-18
13회차 : 2022-03-14 ~2022-08-24
12회차 : 2022-02-15 ~2022-08-04
11회차 : 2022-02-15 ~2022-08-04
10회차 : 2022-01-10 ~2022-06-23
9회차 : 2021-12-01 ~2022-05-17
8회차 : 2021-11-15 ~2022-04-27
7회차 : 2021-09-15 ~2022-02-28
6회차 : 2021-08-25 ~2022-01-28
5회차 : 2021-04-01 ~2021-08-31
4회차 : 2021-03-17 ~2021-08-19
3회차 : 2021-03-15 ~2021-08-13
2회차 : 2020-12-07 ~2021-05-21
1회차 : 2020-11-02 ~2021-04-12
--------------- 회차별 모집인원 ---------------
14회차 : 훈련진행중
13회차 : 훈련진행중
12회차 : 훈련진행중
11회차 : 훈련진행중
10회차 : 25명
9회차 : 25명
8회차 : 25명
7회차 : 25명
6회차 : 24명
5회차 : 25명
4회차 : 25명
3회차 : 25명
2회차 : 28명
1회차 : 28명
--------------- 회차별 [수료, 취업]인원 ---------------
14회차 : 훈련진행중
13회차 : 훈련진행중
12회차 : 훈련진행중
11회차 : 훈련진행중
10회차 : (/)
9회차 : (/)
8회차 : /23
7회차 : 20/23
6회차 :

## 1차 DateFrame 만들기

### Define column

> 기존에서 순서 바꾸고, 중복 제거

순서이렇게 하자

> 기관명, 과정명, 회차, 훈련비, 훈련시작일, 훈련종료일, 훈련시간, NCS 직무분류, 모집인원, 수강확정인원, 수강신청인원, 수강생 평균 연령대(세), 수강생 만족도, 취업률 고용보험가입 3개월, 취업인원(3개월), 수료인원, 취업률 고용보험가입 6개월, 취업인원(6개월)

> column size : 18

- 훈련시간, 회차별 만족도 -> 중복으로 뺌 

In [19]:
col = ['기관명', '과정명', '회차', '훈련비', '훈련시작일', '훈련종료일', 
       '훈련시간', 'NCS 직무분류', '모집인원', '수강확정인원', '수강신청인원', 
       '수강생 평균 연령대(세)', '수강생 만족도', '취업률 고용보험가입 3개월', 
       '취업인원(3개월)', '수료인원','취업률 고용보험가입 6개월', '취업인원(6개월)']
len(col)

18

## 데이터 프레임 열만 생성

In [20]:
df = pd.DataFrame(columns = col)
df

,기관명,과정명,회차,훈련비,훈련시작일,훈련종료일,훈련시간,NCS 직무분류,모집인원,수강확정인원,수강신청인원,수강생 평균 연령대(세),수강생 만족도,취업률 고용보험가입 3개월,취업인원(3개월),수료인원,취업률 고용보험가입 6개월,취업인원(6개월)


## 데이터 추가

> 1. concat : 추가할 대상이 데이터 프레임

> 2. append : 추가할 대상이 딕셔너리 -> 컬럼 이름 : key, 데이터 : value

> 3. loc : 추가할 대상이 리스트 -> 이때 loc에 들어가는 인덱스는 맨 마지막 인덱스로 설정.

일단 뭐든 시도

회차별로 한 번에 뽑을 수 있는 데이터는 따로 모을 예정

### dictionary로 만들어서 DataFrame에 append 해야 겠다.
---

> 각 컬럼 종류별로 리스트로 넣고 join 함수로 str로 바꿔서 dictionary에 넣고 미리 만든 df랑 append로 합치기

> frame.append 메소드가 나중 pandas에서는 사라질거라는 경고문.

> pandas.concat을 쓰라는 권고

> 나중을 위해 loc로 변경

## 2차 크롤링
---
list로 데이터 담기 시작

In [70]:
# 베이스 url
url = 'https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000286660&tracseTme=1&crseTracseSe=C0061&trainstCstmrId=500020018488#undefined'
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')

# columns (중복 일부러 하나 둠)
orga = []# 기관명
cour = []# 과정명
seri = [str(s+1) for s in range(len(soup.select('a.tool_tip_btn')))] # 회차
seri.reverse()
pric = []# 훈련비
prev = [] # 회차별 훈련시작 +
fini = [] # 회차별 훈련종료 +
spen = []# 훈련시간
NCS_dist = []# NCS 직무 분류
recu = [] # 회차별 모집인원 +
conf = [] # 회차별 수강확정인원 +
appl = [] # 회차별 수강신청인원 +
mean_age = []# 수강생 평균 연령대
sati = [] # 회차별 만족도 +
empl_rate_3 = [] # 회차별 취업률(3개월) +
empl_3 = [] # 회차별 취업인원 (3개월) +
comp_3 = [] # 회차별 수료인원 (3개월) +
empl_rate_6 = [] # 회차별 취업률(6개월) +
empl_6 = [] # 회차별 취업인원 (6개월) +
comp_6 = [] # 회차별 수료인원 (6개월) +



# 회차별 만족도
print('-'*15,'회차별 만족도','-'*15)
count = len(soup.select('a.tool_tip_btn')) + 1
for i in soup.select('table.view')[2:]:
    if i.text.find('훈련진행중') != -1:
        count -= 1
        sati.append('훈련진행중')
        print(f'{count}회차 만족도 수집 완료')
    else:
        for j in i.select('td'):
            tableview = j.text.replace('\n','').replace('\t','').strip()
            if re.findall('[.\d]+\s점',tableview) != []:
                count -= 1
                sati.append(re.findall('[.\d]+\s점',tableview).pop().replace(' 점',''))
                print(f"{count}회차 만족도 수집 완료")
print('회차별 만족도 결과')
print(sati)
print(f'size : {len(sati)}')

# 회차별 훈련기간
print('-'*15,'회차별 훈련기간','-'*15)
count = len(soup.select('a.tool_tip_btn')) + 1
for series in soup.select('span.con')[17:]:
    if series.text.find('~') != -1:
        count -= 1
        print(f'{count}회차 훈련 시작일, 종료일 수집 완료')
        prev.append(series.text.replace('\n','').replace('\t','').strip()[:series.text.replace('\n','').replace('\t','').strip().find('~')])
        fini.append(series.text.replace('\n','').replace('\t','').strip()[series.text.replace('\n','').replace('\t','').strip().find('~')+1:])
#         print(series.text.replace('\n','').replace('\t','').strip()[:series.text.replace('\n','').replace('\t','').strip().find('~')])
#         print(series.text.replace('\n','').replace('\t','').strip()[series.text.replace('\n','').replace('\t','').strip().find('~')+1:])
print('회차별 시작일')
print(prev)
print(f'size : {len(prev)}')
print('회차별 종료일')
print(fini)
print(f'size : {len(fini)}')

# 회차별 모집인원
print('-'*15,'회차별 모집인원','-'*15)
count = len(soup.select('a.tool_tip_btn')) + 1
for i in soup.select('table.view')[2:]:
    if i.text.find('훈련진행중') != -1:
        count -= 1
        recu.append('훈련진행중')
        print(f'{count}회차 모집인원 수집 완료')
    else:
        for j in i.select('td'):
            if re.findall('\d+명/',j.text) != []:
                count -= 1
                recu.append(re.findall('\d+명/',j.text).pop().replace('명/',''))
                print(f"{count}회차 : 모집인원 수집 완료")
print('회차별 모집인원')
print(recu)
print(f'size : {len(recu)}')
                      
# 취업률 (고용보험가입 3개월) 취업인원/수료인원
series_count = len(soup.select('a.tool_tip_btn'))
# count = len(soup.select('a.tool_tip_btn')) + 1
cnt = 0
for i in soup.select('table.view'):
    if i.text.find('취업률 고용보험가입 3개월') != -1:
        for j in i.select('b'):
            cnt += 1
            if cnt % 3 == 1:
                rate_empl = j.text.replace('\n','').replace('\t','')
                empl_rate_3.append(rate_empl[:rate_empl.find("%")])
                empl_3.append(rate_empl[rate_empl.find("%")+2:rate_empl.find("/")])
                comp_3.append(rate_empl[rate_empl.find("/")+1:-1])
                print('취업률 수집 완료')
while len(empl_rate_3) != series_count:
    empl_rate_3.insert(0,'훈련진행중')
while len(empl_3) != series_count:
    empl_3.insert(0,'훈련진행중')
while len(comp_3) != series_count:
    comp_3.insert(0,'훈련진행중')
print(empl_rate_3)
print(f'size : {len(empl_rate_3)}')
print(empl_3)
print(f'size : {len(empl_3)}')
print(comp_3)
print(f'size : {len(comp_3)}')


# 취업률 (고용보험가입 6개월) 취업인원/수료인원
series_count = len(soup.select('a.tool_tip_btn'))
# count = len(soup.select('a.tool_tip_btn')) + 1
cnt = 0
for i in soup.select('table.view'):
    if i.text.find('취업률 고용보험가입 6개월') != -1:
        for j in i.select('b'):
            cnt += 1
            if cnt % 3 == 2:
                rate_empl = j.text.replace('\n','').replace('\t','')
                empl_rate_6.append(rate_empl[:rate_empl.find("%")])
                empl_6.append(rate_empl[rate_empl.find("%")+2:rate_empl.find("/")])
                comp_6.append(rate_empl[rate_empl.find("/")+1:-1])
                
while len(empl_rate_6) != series_count:
    empl_rate_6.insert(0,'훈련진행중')
while len(empl_6) != series_count:
    empl_6.insert(0,'훈련진행중')
while len(comp_6) != series_count:
    comp_6.insert(0,'훈련진행중')
print(empl_rate_6)
print(f'size : {len(empl_rate_6)}')
print(empl_6)
print(f'size : {len(empl_6)}')
print(comp_6)
print(f'size : {len(comp_6)}')

# 회차별 수강확정인원, 수강신청인원
print('-'*15,'회차별 수강 [확정,신청] 인원','-'*15)
for a in soup.select('table.view'):
    for b in a.select('td'):
        if b.text.find('수강확정인원') != -1:
#             print(re.findall('\d+',b.text))
            conf.append(re.findall('\d+',b.text)[0])
            appl.append(re.findall('\d+',b.text)[1])
print('수강확정인원')
print(conf)
print(f'size : {len(conf)}')
print('수강신청인원')
print(appl)
print(f'size : {len(appl)}')

count = len(soup.select('a.tool_tip_btn')) + 1
    
for patterns in soup.select('a.tool_tip_btn'):
    series_url = f'https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000286660&tracseTme={index+1}&crseTracseSe=C0061&trainstCstmrId=500020018488#undefined'
    resp_series = requests.get(series_url)
    soup_series = BeautifulSoup(resp_series.text, "html.parser")

    count -= 1
    print(f'---------------- {count} 회차 ---------------')
    
    # 차단 방지 시간 딜레이
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(5)
    time.sleep(a)
    
    # 기관명
    for organ in soup_series.select('p.add'):
        orga.append(organ.text.strip())

    # 과정명
    for course in soup_series.select('h4.tit'):
#         print(course.text.replace('\n','').replace('\t',''))
        cour.append(course.text.replace('\n','').replace('\t',''))
    
    # 훈련비
    pric.append(soup.select('p.price')[0].text[:soup.select('p.price')[0].text.find(' 원')].replace('\n', '').replace('\t', ''))
#     for price in soup_series.select('p.price'):
#         print(price.text.strip().replace('\n','')[:price.text.find('원')].strip().replace('\t',''))
#         pric.append(price.text.strip().replace('\n','')[:price.text.find('원')].strip().replace('\t',''))

    # 훈련시간
    for i in soup.select('span.con'):
        if i.text.find('일') != -1:
#             print(i.text.replace('\n','').replace('\t',''))
            spen.append(i.text.replace('\n,','').replace('\t','').strip().replace('\n',''))
    # NCS 자격분류
    for NCS in soup_series.select('span.con'):
        if re.findall('\d{6,}',NCS.text) != []:
#             print(re.findall('.+\d{6,}.',NCS.text).pop().replace('\t',''))
            NCS_dist.append(re.findall('.+\d{6,}.',NCS.text).pop().replace('\t',''))
    # 수강생 평균나이
    for g in soup.select('span.con'):
        if g.text.find('세') != -1:
#         print(re.findall("\d+\s세", g.text).pop().replace(' 세',''))
            mean_age.append(re.findall("\d+\s세", g.text).pop().replace(' 세',''))

print(orga)
print(f'size : {len(orga)}')
print(cour)
print(f'size : {len(cour)}')
print(pric)
print(f'size : {len(pric)}')
print(spen)
print(f'size : {len(spen)}')
print(NCS_dist)
print(f'size : {len(NCS_dist)}')

--------------- 회차별 만족도 ---------------
14회차 만족도 수집 완료
13회차 만족도 수집 완료
12회차 만족도 수집 완료
11회차 만족도 수집 완료
10회차 만족도 수집 완료
9회차 만족도 수집 완료
8회차 만족도 수집 완료
7회차 만족도 수집 완료
6회차 만족도 수집 완료
5회차 만족도 수집 완료
4회차 만족도 수집 완료
3회차 만족도 수집 완료
2회차 만족도 수집 완료
1회차 만족도 수집 완료
회차별 만족도 결과
['훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '75.2', '82.4', '90.84', '90', '91.58', '84.54', '89.6', '89.56', '80.74', '95.56']
size : 14
--------------- 회차별 훈련기간 ---------------
14회차 훈련 시작일, 종료일 수집 완료
13회차 훈련 시작일, 종료일 수집 완료
12회차 훈련 시작일, 종료일 수집 완료
11회차 훈련 시작일, 종료일 수집 완료
10회차 훈련 시작일, 종료일 수집 완료
9회차 훈련 시작일, 종료일 수집 완료
8회차 훈련 시작일, 종료일 수집 완료
7회차 훈련 시작일, 종료일 수집 완료
6회차 훈련 시작일, 종료일 수집 완료
5회차 훈련 시작일, 종료일 수집 완료
4회차 훈련 시작일, 종료일 수집 완료
3회차 훈련 시작일, 종료일 수집 완료
2회차 훈련 시작일, 종료일 수집 완료
1회차 훈련 시작일, 종료일 수집 완료
회차별 시작일
['2022-05-02 ', '2022-03-14 ', '2022-02-15 ', '2022-02-15 ', '2022-01-10 ', '2021-12-01 ', '2021-11-15 ', '2021-09-15 ', '2021-08-25 ', '2021-04-01 ', '2021-03-17 ', '2021-03-15 ', '2020-12-07 ', '2020-11-02 ']
size : 14
회차별 종료일
['2022-10-18', '2022-08-24

### list -> str -> dictionary 로 변환

In [170]:
li_list =[orga,cour,seri,pric,prev,fini,spen,NCS_dist,recu,conf,appl,mean_age,sati,empl_rate_3,empl_3,comp_3,empl_rate_6,empl_6]


['스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원']
['빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감']
['14', '13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '2', '1']
['13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240']
['2022-05-02 ', '2022-03-14 ', '2022-02-15 ', '2022-02-15 ', '2022-01-10 ', '2021-12-01 ', '2021-11-15 ', '2021-09-15 ', '2021-08-25 ', '2021-04-01 ', 

> list to str

### dictionary -> DataFrame 변환

In [172]:
# pd.DataFrame(columns = colu)
df = pd.DataFrame(columns = col)
df

,기관명,과정명,회차,훈련비,훈련시작일,훈련종료일,훈련시간,NCS 직무분류,모집인원,수강확정인원,수강신청인원,수강생 평균 연령대(세),수강생 만족도,취업률 고용보험가입 3개월,취업인원(3개월),수료인원,취업률 고용보험가입 6개월,취업인원(6개월)


In [259]:
# df_1 = pd.DataFrame([seri])
# df_2 = pd.DataFrame([pric])
# pd.concat([df_1,df_2], ignore_index=True)
# df_list = []
# for li in li_list:
#     df_list.append(pd.DataFrame([li]))
# pd.concat(df_list, ignore_index=True)
var_list = ['orga','cour','seri','pric','prev','fini','spen','NCS_dist','recu','conf','appl','mean_age','sati','empl_rate_3','empl_3','comp_3','empl_rate_6','empl_6']

c_count = 0

pot2 = []
for ind in range(len(seri)):
    pot1 = []
    for li in li_list:
        pot1.append(li[ind])
#     print(pot1)
    pot2.append(pot1)

[['스마트인재개발원', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '14', '13,762,240', '2022-05-02 ', '2022-10-18', '116일,총928시간', '응용SW엔지니어링(20010202)', '훈련진행중', '25', '25', '27', '훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중'], ['스마트인재개발원', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '13', '13,762,240', '2022-03-14 ', '2022-08-24', '116일,총928시간', '응용SW엔지니어링(20010202)', '훈련진행중', '25', '25', '27', '훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중'], ['스마트인재개발원', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '12', '13,762,240', '2022-02-15 ', '2022-08-04', '116일,총928시간', '응용SW엔지니어링(20010202)', '훈련진행중', '25', '25', '27', '훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중'], ['스마트인재개발원', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '11', '13,762,240', '2022-02-15 ', '2022-08-04', '116일,총928시간', '응용SW엔지니어링(20010202)', '훈련진행중', '25', '25', '27', '훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중'], ['스마트인재개발원', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '10', '13,762,240', '2022-01-10 ', '2022-06-23', '116일,총928시간', '응용SW엔지니어링(20010202)', '25', '25', '25', '27', '7

In [282]:
li_list

[['스마트인재개발원',
  '스마트인재개발원',
  '스마트인재개발원',
  '스마트인재개발원',
  '스마트인재개발원',
  '스마트인재개발원',
  '스마트인재개발원',
  '스마트인재개발원',
  '스마트인재개발원',
  '스마트인재개발원',
  '스마트인재개발원',
  '스마트인재개발원',
  '스마트인재개발원',
  '스마트인재개발원'],
 ['빅데이터 분석서비스 개발자과정(NCS)모집마감',
  '빅데이터 분석서비스 개발자과정(NCS)모집마감',
  '빅데이터 분석서비스 개발자과정(NCS)모집마감',
  '빅데이터 분석서비스 개발자과정(NCS)모집마감',
  '빅데이터 분석서비스 개발자과정(NCS)모집마감',
  '빅데이터 분석서비스 개발자과정(NCS)모집마감',
  '빅데이터 분석서비스 개발자과정(NCS)모집마감',
  '빅데이터 분석서비스 개발자과정(NCS)모집마감',
  '빅데이터 분석서비스 개발자과정(NCS)모집마감',
  '빅데이터 분석서비스 개발자과정(NCS)모집마감',
  '빅데이터 분석서비스 개발자과정(NCS)모집마감',
  '빅데이터 분석서비스 개발자과정(NCS)모집마감',
  '빅데이터 분석서비스 개발자과정(NCS)모집마감',
  '빅데이터 분석서비스 개발자과정(NCS)모집마감'],
 ['14', '13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '2', '1'],
 ['13,762,240',
  '13,762,240',
  '13,762,240',
  '13,762,240',
  '13,762,240',
  '13,762,240',
  '13,762,240',
  '13,762,240',
  '13,762,240',
  '13,762,240',
  '13,762,240',
  '13,762,240',
  '13,762,240',
  '13,762,240'],
 ['2022-05-02 ',
  '2022-03-14 ',
  '2022-02-15 ',
  '2022-02-15 

In [284]:
var_list = ['orga','cour','seri','pric','prev','fini','spen','NCS_dist','recu','conf','appl','mean_age','sati','empl_rate_3','empl_3','comp_3','empl_rate_6','empl_6']

for li, cols in zip(li_list, df.columns.tolist()):
    df[cols] = li
df

,기관명,과정명,회차,훈련비,훈련시작일,훈련종료일,훈련시간,NCS 직무분류,모집인원,수강확정인원,수강신청인원,수강생 평균 연령대(세),수강생 만족도,취업률 고용보험가입 3개월,취업인원(3개월),수료인원,취업률 고용보험가입 6개월,취업인원(6개월)
0,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,14,"13,762,240",2022-05-02,2022-10-18,"116일,총928시간",응용SW엔지니어링(20010202),훈련진행중,25,25,27,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
1,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,13,"13,762,240",2022-03-14,2022-08-24,"116일,총928시간",응용SW엔지니어링(20010202),훈련진행중,25,25,27,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
2,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,12,"13,762,240",2022-02-15,2022-08-04,"116일,총928시간",응용SW엔지니어링(20010202),훈련진행중,25,25,27,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
3,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,11,"13,762,240",2022-02-15,2022-08-04,"116일,총928시간",응용SW엔지니어링(20010202),훈련진행중,25,25,27,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
4,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,10,"13,762,240",2022-01-10,2022-06-23,"116일,총928시간",응용SW엔지니어링(20010202),25,25,25,27,75.2,,,,,
5,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,9,"13,762,240",2021-12-01,2022-05-17,"116일,총928시간",응용SW엔지니어링(20010202),25,25,25,27,82.4,,,,,
6,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,8,"13,762,240",2021-11-15,2022-04-27,"116일,총928시간",응용SW엔지니어링(20010202),25,25,25,27,90.84,60.9,14,23,,
7,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,7,"13,762,240",2021-09-15,2022-02-28,"116일,총928시간",응용SW엔지니어링(20010202),25,25,25,27,90,87,20,23,,
8,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,6,"13,762,240",2021-08-25,2022-01-28,"116일,총928시간",응용SW엔지니어링(20010202),24,24,24,27,91.58,63.2,12,19,84.2,16
9,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,5,"13,762,240",2021-04-01,2021-08-31,"116일,총928시간",응용SW엔지니어링(20010202),25,25,25,27,84.54,0,0,18,83.3,15
